# Utilities

```julia
import Pkg;

Pkg.add("PyCall")
Pkg.add("DataFrames")
Pkg.add("XLSX")
Pkg.add("HDF5")
Pkg.add("JLD")
Pkg.add("JSON")

Pkg.add("LaTeXStrings")
```

In [1]:
using Printf
using Dates


function genLoading(nmax::Int, leg::Array{Float64}=[0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 0.9, 0.99, 1])
    #=
    utilitary function for showing estimated time remaining
    =#
    t0 = now();
    lt = now();
    
    function loading(i::Int)
        perc = i/nmax;
        f = abs.(leg.-perc) .< 1/nmax;
        t1 = now();
        et =  round(t1-lt, Dates.Second).value;
        if (sum(f) >= 1) || (et > 600) || (i==1)
            elapsed = t1-t0;
            period = Dates.canonicalize(Dates.CompoundPeriod(elapsed));
            rem = Dates.canonicalize(Dates.CompoundPeriod(round((nmax-i)/i)*elapsed));
            @printf("Loading... %3.1f%% \t (%s) - (%s)\n", perc*100, period, rem);
            lt = now();
        end
        return nothing;
    end
    return loading;
end

genLoading (generic function with 2 methods)

## Construct policies

The state space is given by the set of $l+1$-tuples 
$$\mathbb{X}:=\{(i,s_0,s_1,\ldots,s_l)| i \in\{0,1,\ldots,k\}, s_j \in \{0,1\}, j=0,1,\ldots,l\}.$$
where 

* $i$ denotes patient type,
* $k$ is the number of types of patients,
* $s_j$ denotes the state of facility $j$, 
* $l$ is the number of facilities.

In [1]:
function lenStateSpace(k::Int, l::Int)
    #=
    Returns the size of the state space of the system
    k: denotes the number of patients
    l: denotes the number of facilities
    =#
    lenS = (k+1)*(2^(l+1));
    return lenS;
end

function genStateSpace(k::Int, l::Int)
    #=
    Generates the state space 𝕏 of the system
    k: denotes the number of patients
    l: denotes the number of facilities
    =#
    return vec(collect(Iterators.product(0:k, [0:1 for j in 0:l]...)));
end

genStateSpace (generic function with 1 method)

In [2]:
function genR(readmissions::Array{Array{Float64}})
    function r(x::Array{Int}, j::Int)
        #=
        This function takes a vector state x=[i, s_0, s_1, ..., s_l],
        where i is the type of patient, and s_k is the state of facility k for all k,
        and a number 0<=j<=l.
        
        This function returns the cost of readmission for patient i in snf j.
        =#
        # first number in the array is type of patient
        i = x[1];
        if i == 0
            # patient type 0 represents no patient at this time, thus, cost is always 0
            return 0;
        end
        if j == 0
            # facility type 0 represents the event "send patient home", and since this option
            # is undesirable, the readmission cost is set very high
            return 100;
        end
        return readmissions[j][i];
    end
    return r;
end

genR (generic function with 1 method)

In [ ]:
function genP(λ::Function, pMat::Function, l::Int)
    function p(x1::Array{Int}, x0::Array{Int}, a::Int)
        #=
        This function returns the probability P_{X0, X1}^{a} defined
        as the probability of transitioning from one state x0 to a state
        x1 for SNF a
        =#
        return λ(x1[1])*prod([pMat(a, j, x0[j+2], x1[j+2]) for j in 0:l]);
    end
    return p;
end

the function "constructMatrices" below defines the transition probability matrices

\begin{align*}
\mathbf{P}^{0,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&&\dots&\mathbf{P}^{3,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix}\\
\mathbf{P}^{0,1} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,1} &= \begin{bmatrix}
p^{1, 1}_{1, 1}&1-p^{1, 1}_{1, 1}\\
p^{1, 1}_{2, 1}&1-p^{1, 1}_{2, 1}
\end{bmatrix},& &\dots&\mathbf{P}^{l,1} &= \begin{bmatrix}
p^{l, 1}_{1, 1}&1-p^{l, 1}_{1, 1}\\
p^{l, 1}_{2, 1}&1-p^{l, 1}_{2, 1}
\end{bmatrix}\\
&\vdots&&\vdots&&\ddots&&\vdots\\
\mathbf{P}^{0,l} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,l} &= \begin{bmatrix}
p^{1, l}_{1, 1}&1-p^{1, l}_{1, 1}\\
p^{1, 1}_{2, 1}&1-p^{1, l}_{2, 1}
\end{bmatrix},&&\dots&\mathbf{P}^{l,l} &= \begin{bmatrix}
p^{l, l}_{1, 1}&1-p^{l, l}_{1, 1}\\
p^{l, l}_{2, 1}&1-p^{l, l}_{2, 1}
\end{bmatrix}
\end{align*}

In [ ]:
function constructMatrices(vec)
    model_p_ce = [
        [[vec[i], 1-vec[i]], [vec[i+1], 1-vec[i+1]]] for i in 1:2:length(vec)
    ]
    return model_p_ce
end

In [ ]:
function genA(l::Int, genS)
    function A(x::Array{Int})
        #=
        Let x=[i, s1, s2, ...sl] be the state of the system,
        where sk is the state of the SNF k (available:1, unavailable:0).
        This function returns the set of available SNFs given the
        state of the system, defined as the set {k: sk=1}.
        If there's no viable options, then, returns the SNF 0,
        which sends patient home
        =#
        if x[1] == 0
            return [0];
        end
        mA = 2:l+1;
        ls = [y-1 for y in mA if x[y+1] == 1];
        if isempty(ls)
            ls = [0];
        end
        return ls;
    end
    Aev = Dict(x => A(x) for x in genS);
    return function (x) return Aev[x] end;
end

## Policies

In [ ]:
function genMyopicPolicy(A::Function, r::Function, λ::Function, genS)
    #=
    This function generates the myopic policy
    =#
    function f(x)
        s = [r(x, a) for a in A(x)];
        return A(x)[argmin(s)];
    end
    Policy = Dict(x => f(x) for x in genS);
    return Policy;
end

In [ ]:
function genHeuristic2(r::Function, A::Function, genS::Array{Array{Int}})
    #=
    This function generates the heuristic 2 policy
    =#
    rCache = Dict(xp => min([r(xp, ap) for ap in A(xp)]...) for xp in genS);
    function heur2(λ::Function, p::Function)
        function compare(x, xp, a)
            rMin = rCache[xp];
            return p(xp, x, a)*rMin;
        end
        function order(x, a)
            res = sum([compare(x, xp, a) for xp in genS]);
            return r(x, a) + res;
        end
        return Dict(x => A(x)[argmin([order(x, a) for a in A(x)])] for x in genS);
    end
    return heur2;
end

For heuristic 3, since it is computationally expensive, then we first create a "cache" dictionary with the two-step ahead costs, defined in the function "genTransitionProbabilityCache" below, and then use that to evaluate the third heuristic using the function "genHeuristic3" below

In [ ]:
function genTransitionProbabilityCache(lenS::Int, genS::Array{Array{Int}}, p::Function, n::Int, l::Int)
    P = Dict(a => [p(genS[j], genS[i], a) for i in 1:lenS, j in 1:lenS] for a in 0:l);
    cachePn = Dict(k => Dict(a => P[a]^k for a in 0:l) for k in 1:n);
    cacheEv = Dict();
    cacheInd = Dict(a => i for (a, i) in zip(genS, 1:lenS));
    for k in 1:n
        function func(x2, x0, a)
            idx0 = cacheInd[hcat(x0...)];
            idx2 = cacheInd[hcat(x2...)];
            return cachePn[k][a][idx0, idx2];
        end
        cacheEv[k] = func;
    end
    return cacheEv;
end

function genHeuristic3(r::Function, A::Function, genS::Array{Array{Int}}, lenS::Int, l::Int; ran_n::Array{Int}=[1], ran_w::Array{Int}=[1])
    rCache = Dict(xp => min([r(xp, ap) for ap in A(xp)]...) for xp in genS);
    function heur3(λ, p)
        cachePn = genTransitionProbabilityCache(lenS, genS, p, max(ran_n...), l);
        function compare(x::Array{Int}, xp::Array{Int}, a::Int; k::Int=1)
            rMin = rCache[xp];
            return cachePn[k](xp, x, a)*rMin;
        end
        function order(x::Array{Int}, a::Int, n::Int, w::Int)
            res = sum(sum([compare(x, xp, a, k=k)*(w^k) for xp in genS]) for k in 1:n);
            return r(x, a) + res;
        end
        
        pols = [];
        for n in ran_n
            pols_n = [];
            for w in ran_w
                policy = Dict(x => A(x)[argmin([order(x, a, n, w) for a in A(x)])] for x in genS);
                push!(pols_n, policy);
            end
            push!(pols, pols_n);
        end
        return pols;
    end
    return heur3;
end

# Data

In [5]:
# Matrix with readmission rates defined in the paper
readmissions = Array{Array{Float64}}([
    [14.3,  9.5, 19.1, 19.2],
    [16.4, 12.8, 20.1, 20.4],
    [15.6, 12.4, 20.6, 20.2],
    [9.1,   8.7, 11.2, 19.6],
    [20.6,  5.8, 19.0, 13.4],
])

r = genR(readmissions);

r (generic function with 1 method)

# Policy evaluation

In [ ]:
function hImprovingAr(h_n, g_n, lenS, genS, A, r, p)
    function tradeOffAr(x0, a)
        return r(x0, a) - g_n + sum(p(genS[i], x0, a)*h_n[i] for i in 1:lenS);
    end
    
    d_np1 = Dict();
    for s in genS
        As = A(s);
        a_opt = As[argmin([trade_off_ar(s, a) for a in As])];
        d_np1[s] = a_opt;
    end
    return d_np1;
end

In [ ]:
using LinearAlgebra


function genTransitionMatrix(d, genS, p)
    P_d = [p(xj, xi, d[xi]) for xi in genS, xj in genS];
    return P_d;
end

function evalPolicyAr(d, genS, r, p, ind=Nothing)
    P_d = genTransitionMatrix(d, genS, p);
    r_d = [r(x, d[x]) for x in genS];
    
    n = size(P_d, 1);
    if ind == Nothing
        ind = rand(1:n);
    end

    M = (1.0I - P_d);
    M[:, ind] .= 1;
    invM = inv(M);
    z = invM*r_d;
    g = z[ind];
    z[ind] = 0;
    h = z;
    return h, g;
end

In [8]:
function genPπ(A::Function, π::Function, genS, r::Function, p::Function)
    lenS = length(genS);
    
    Pπ = zeros(lenS, lenS);
    @sync begin
        for (i, xi) in enumerate(genS), (j, xj) in enumerate(genS)
#             Threads.@spawn begin
            begin
                Pπ[i, j] = sum([p(xj, xi, a)*π(a, xi) for a in A(xi)]);
            end
        end
    end
    
    πr = zeros(lenS);
    @sync begin
        for (i, xi) in enumerate(genS)
#             Threads.@spawn begin
            begin
                πr[i] = sum([π(a, xi)*r(xi, a) for a in A(xi)]);
            end
        end
    end
    
    return Pπ, πr;
end

function evaluatePolicyARRandomPolicy(Pπ::Array{Float64, 2}, πr::Vector{Float64}; ind=Nothing)
    n = size(Pπ, 1);
    if ind == Nothing
        ind = rand(1:n);
    end

    M = (1.0I - Pπ);
    M[:, ind] .= 1;
    invM = inv(M);
    z = invM*πr;
    g = z[ind];
    z[ind] = 0;
    h = z;
    return h, g;
end

evaluatePolicyARRandomPolicy (generic function with 1 method)

In [ ]:
function policyIterationAr(d_0, lenS, genS, A, r, p; n_max=100, verbose=false)
    h_n = Nothing;
    g_n = Nothing;
    d_n = d_0;
    
    for it in 1:n_max
        h_n, g_n = evalPolicyAr(d_n, genS, r, p);
        d_np1 = hImprovingAr(h_n, g_n, lenS, genS, A, r, p);
        if d_n == d_np1
            if verbose
                println("\nConverged (iteration={})".format(it));
            end
            break;
        end
        d_n = d_np1;
    end
    return d_n, h_n, g_n;
end

In [ ]:
function runAllHeuristics(heur2, lenS, genS, A, r, λ, p; heur3=nothing)
    mPolicyOpt = myopic_policy(A, r, λ, genS);
    h2PolicyOpt = heur2(λ, p);
    _, gmOpt = evalPolicyAr(mPolicyOpt, genS, r, p);
    _, gh2Opt = evalPolicyAr(h2PolicyOpt, genS, r, p);
    
    optPol, _, g = policy_iteration_ar(h2PolicyOpt, lenS, genS, A, r, p);
    d = Dict()
    
    d["pol_opt"] = optPol
    d["pol_myopic"] = mPolicyOpt
    d["pol_heur2"] = h2PolicyOpt
    
    d["g_opt"] = g
    d["g_myopic"] = gmOpt
    d["g_heur2"] = gh2Opt
    
    d["g/g_myopic"] = g/gmOpt
    d["g/g_heur2"] = g/gh2Opt
    
    d["g_myopic/g_heur2"] = gmOpt/gh2Opt
    
    d["p"] = p
    
    if heur3 != nothing
        h3PolicyOpt = heur3(λ, p);
        d["pol_heur3"] = []
        d["g_heur3"] = []
        d["g/g_heur3"] = []
        d["g_myopic/g_heur3"] = []
        for row in h3PolicyOpt
            cols = []
            gs = []
            opt_gs = []
            myo_gs = []
            for col in row
                _, gh3Opt = evaluate_policy_ar(col, genS, r, p);
                push!(cols, col)
                push!(gs, gh3Opt)
                push!(opt_gs, g/gh3Opt)
                push!(myo_gs, gmOpt/gh3Opt)
            end
            push!(d["pol_heur3"], cols)
            push!(d["g_heur3"], gs)
            push!(d["g/g_heur3"], opt_gs)
            push!(d["g_myopic/g_heur3"], myo_gs)
        end
    end
    return d
end